### Manipulation of PEAKS de novo results of Trocas 7 (high water, April 2019) incubation samples LC-MS/MS data using python.

Starting with:

    PEAKS de novo results (.csv) of PTM-optimized sequencing >50% ALC

Goal:

    Files with stripped (no PTMs) peptide lists and
    Columns with #'s of each modification in every sequence
    Column with stripped peptide lengths (# amino acids)
    
### To use:

#### 1. Change the input file name in *IN 4*
#### 2. Use 'find + replace' (Esc + F) to replace the TROCAS # (e.g., 101) for another
#### 2. Change output file name in *IN 6*, *IN 7*, *IN 8*

In [1]:
# LIBRARIES
#import pandas library for working with tabular data
import os
os.getcwd()
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import kde
#import regular expresson (regex)
import re
#check pandas version
pd.__version__

'1.0.5'

In [2]:
cd /home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/T7-INC-UWPR_Apr2021_Fus_T00T24/DN50_10lgP15/410_TROCAS7_Fusion_Apr2021_DENOVO_134/

/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/T7-INC-UWPR_Apr2021_Fus_T00T24/DN50_10lgP15/410_TROCAS7_Fusion_Apr2021_DENOVO_134


In [4]:
# read the CSVs into a dataframe using the read_csv function and call 'peaks'

peaks410 = pd.read_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/T7-INC-UWPR_Apr2021_Fus_T00T24/DN50_10lgP15/410_TROCAS7_Fusion_Apr2021_DENOVO_134/410_TROCAS7_Fusion_Apr2021_DENOVO_134_DN50.csv")

print("# redundant Peaks peptides >50% ALC in combined dataframe:", len(peaks410))

print(peaks410.columns)

# These columns mess things up- get rid of them

del peaks410['Fraction']
del peaks410['Scan']
del peaks410['Source File']
del peaks410['Tag Length']
del peaks410['PTM']
del peaks410['tag (>=0%)']
del peaks410['mode']
del peaks410['local confidence (%)']


columns = ['Peptide', 'ALC', 'length', 'm/z', 'z', 'RT', 'Area',
       'Mass', 'ppm']

peaks410.columns = columns

mean_len = peaks410['length'].mean()
print(mean_len)

# look at the dataframe
peaks410.head()

# redundant Peaks peptides >50% ALC in combined dataframe: 236
Index(['Fraction', 'Scan', 'Source File', 'Peptide', 'Tag Length', 'ALC (%)',
       'length', 'm/z', 'z', 'RT', 'Area', 'Mass', 'ppm', 'PTM',
       'local confidence (%)', 'tag (>=0%)', 'mode'],
      dtype='object')
9.639830508474576


,Peptide,ALC,length,m/z,z,RT,Area,Mass,ppm
0,SPATLNSR,98,8,423.2274,2,26.10,446000.0,844.4402,0.0
1,TEELDR,98,6,381.6851,2,24.54,16400.0,761.3555,0.0
2,SC(+57.02)ELSC(+57.02)K,97,7,442.1863,2,23.52,115000.0,882.3575,0.6
3,SPATLNSR,97,8,423.2275,2,26.60,446000.0,844.4402,0.2
4,KM(+15.99)LNAAK,97,7,396.2239,2,44.63,147000.0,790.4371,-4.9


The peptide column has the masses of modifications (e.g., 57.02 Da for carbamidomethylation of cysteine). We want to make new columns with all that information and make a column with only the 'stripped' peptide sequence that's just amino acids - this we can then align against other sequences, for example.

Modified residues were allowed for:

    fixed carbamodimethylation of cysteine 57.021464 C
    varialbe oxidation of methionine: 15.9949 M
    variable deamidation of asparagine, glumatine: 0.984016 NQ

We'll then write this manipulated dataframe to a new file.

In [5]:
# use a count function to enumerate the # of A's (alanines) in each peptide
peaks410['A'] = peaks410['Peptide'].str.count("A")

# use a count function to enumerate the # of C's (cysteines) in each peptide
peaks410['C'] = peaks410['Peptide'].str.count("C")

# use a count function to enumerate the # of D's (aspartic acids) in each peptide
peaks410['D'] = peaks410['Peptide'].str.count("D")

# use a count function to enumerate the # of E's (glutamic acids) in each peptide
peaks410['E'] = peaks410['Peptide'].str.count("E")

# use a count function to enumerate the # of F's (phenylalanines) in each peptide
peaks410['F'] = peaks410['Peptide'].str.count("F")

# use a count function to enumerate the # of G's (glycines) in each peptide
peaks410['G'] = peaks410['Peptide'].str.count("G")

# use a count function to enumerate the # of H's (histidines) in each peptide
peaks410['H'] = peaks410['Peptide'].str.count("H")

# use a count function to enumerate the # of I's (isoleucines) in each peptide
# in peaks410 output, there will be no isoleucines (they're lumped in with leucines)
peaks410['I'] = peaks410['Peptide'].str.count("I")

# use a count function to enumerate the # of K's (lysines) in each peptide
peaks410['K'] = peaks410['Peptide'].str.count("K")

# use a count function to enumerate the # of L's (leucines) in each peptide
# also these include the isoleucines
peaks410['L'] = peaks410['Peptide'].str.count("L")

# use a count function to enumerate the # of M's (methionines) in each peptide
peaks410['M'] = peaks410['Peptide'].str.count("M")

# use a count function to enumerate the # of N's (asparagines) in each peptide
peaks410['N'] = peaks410['Peptide'].str.count("N")

# use a count function to enumerate the # of P's ([prolines]) in each peptide
peaks410['P'] = peaks410['Peptide'].str.count("P")

# use a count function to enumerate the # of Q's (glutamines) in each peptide
peaks410['Q'] = peaks410['Peptide'].str.count("Q")

# use a count function to enumerate the # of R's (arginines) in each peptide
peaks410['R'] = peaks410['Peptide'].str.count("R")

# use a count function to enumerate the # of S's (serines) in each peptide
peaks410['S'] = peaks410['Peptide'].str.count("S")

# use a count function to enumerate the # of T's (threonines) in each peptide
peaks410['T'] = peaks410['Peptide'].str.count("T")

# use a count function to enumerate the # of V's (valines) in each peptide
peaks410['V'] = peaks410['Peptide'].str.count("V")

# use a count function to enumerate the # of W's (tryptophans) in each peptide
peaks410['W'] = peaks410['Peptide'].str.count("W")

# use a count function to enumerate the # of Y's (tyrosines) in each peptide
peaks410['Y'] = peaks410['Peptide'].str.count("Y")

# use a count function to enumerate the # of carbamidomethylated C's in each peptide
peaks410['c-carb'] = peaks410['Peptide'].str.count("57.02")

# use a count function to enumerate the # of oxidized M's in each peptide
peaks410['m-oxid'] = peaks410['Peptide'].apply(lambda x: x.count('M(+15.99)'))

# use a lamba function to enumerate the # of deamidated N's in each peptide
peaks410['n-deam'] = peaks410['Peptide'].apply(lambda x: x.count('N(+.98)'))

# use a count function to enumerate the # of deamidated Q's in each peptide
peaks410['q-deam'] = peaks410['Peptide'].apply(lambda x: x.count('Q(+.98)'))

# create a column with 'stripped' peptide sequences using strip
peaks410['stripped_peptide'] = peaks410['Peptide'].str.replace(r"\(.*\)","")

# add a column with the stripped peptide length (number of AAs)
peaks410['stripped_length'] = peaks410['stripped_peptide'].apply(len)

# total the number of modifications in sequence
peaks410['ptm-total'] = peaks410['c-carb'] + peaks410['m-oxid'] + peaks410['n-deam'] + peaks410['q-deam']

# calculate NAAF numerator for each peptide k
peaks410['NAAF_num.'] = peaks410['Area'] / peaks410['stripped_length']

# write modified dataframe to new txt file, same name + 'stripped'
peaks410.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDN/410_MS_T24_GD_DN/410A_MS_T24_GD_DN50.csv")

# check out the results
peaks410.head()

,Peptide,ALC,length,m/z,z,RT,Area,Mass,ppm,A,...,W,Y,c-carb,m-oxid,n-deam,q-deam,stripped_peptide,stripped_length,ptm-total,NAAF_num.
0,SPATLNSR,98,8,423.2274,2,26.10,446000.0,844.4402,0.0,1,...,0,0,0,0,0,0,SPATLNSR,8,0,55750.000000
1,TEELDR,98,6,381.6851,2,24.54,16400.0,761.3555,0.0,0,...,0,0,0,0,0,0,TEELDR,6,0,2733.333333
2,SC(+57.02)ELSC(+57.02)K,97,7,442.1863,2,23.52,115000.0,882.3575,0.6,0,...,0,0,2,0,0,0,SCK,3,2,38333.333333
3,SPATLNSR,97,8,423.2275,2,26.60,446000.0,844.4402,0.2,1,...,0,0,0,0,0,0,SPATLNSR,8,0,55750.000000
4,KM(+15.99)LNAAK,97,7,396.2239,2,44.63,147000.0,790.4371,-4.9,2,...,0,0,0,1,0,0,KMLNAAK,7,1,21000.000000


In [6]:
# keep only stripped peptide I/L and NAAF
dn_410 = peaks410[['stripped_peptide', 'Area', 'NAAF_num.']]

dn_410.set_index('stripped_peptide')

# write modified dataframe to new txt file
dn_410.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/I-L_NAAFs/410A_MS_T24_GD_DN50_ILnaafs.csv")

dn_410.head()

,stripped_peptide,Area,NAAF_num.
0,SPATLNSR,446000.0,55750.000000
1,TEELDR,16400.0,2733.333333
2,SCK,115000.0,38333.333333
3,SPATLNSR,446000.0,55750.000000
4,KMLNAAK,147000.0,21000.000000


In [7]:
# made a new dataframe that contains the sums of certain columns 
# in the stripped peptide dataframe above (for >50% ALC)

index = ['sample total']

data = {'A': peaks410['A'].sum(),
        'C': peaks410['C'].sum(),
        'D': peaks410['D'].sum(),
        'E': peaks410['E'].sum(),
        'F': peaks410['F'].sum(),
        'G': peaks410['G'].sum(),
        'H': peaks410['H'].sum(),
        'I': peaks410['I'].sum(),
        'K': peaks410['K'].sum(),
        'L': peaks410['L'].sum(),
        'M': peaks410['M'].sum(),
        'N': peaks410['N'].sum(),
        'P': peaks410['P'].sum(),
        'Q': peaks410['Q'].sum(),
        'R': peaks410['R'].sum(),
        'S': peaks410['S'].sum(),
        'T': peaks410['T'].sum(),
        'V': peaks410['V'].sum(),
        'W': peaks410['W'].sum(),
        'Y': peaks410['Y'].sum(),
        'c-carb': peaks410['c-carb'].sum(),
        'm-oxid': peaks410['m-oxid'].sum(),
        'n-deam': peaks410['n-deam'].sum(),
        'q-deam': peaks410['q-deam'].sum(),
        'Total area': peaks410['Area'].sum(),
        'Total length': peaks410['stripped_length'].sum()
       }

totalpeaks410 = pd.DataFrame(data, columns=['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', \
                                            'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', \
                                            'c-carb', 'm-oxid', 'n-deam', 'q-deam', \
                                            'Total area', 'Total length'], index=index)

# calculate percentage of C's with carb (should be 1.0)
totalpeaks410['% C w/ carb'] = totalpeaks410['c-carb'] / totalpeaks410['C'] 

# calculate percentage of M's that are oxidized
totalpeaks410['% M w/ oxid'] = totalpeaks410['m-oxid'] / totalpeaks410['M'] 

# calculate percentage of N's that are deamidated
totalpeaks410['% N w/ deam'] = totalpeaks410['n-deam'] / totalpeaks410['N'] 

# calculate percentage of N's that are deamidated
totalpeaks410['% Q w/ deam'] = totalpeaks410['q-deam'] / totalpeaks410['Q'] 

# calculate NAAF denominator for all peptides in dataset i
totalpeaks410['NAAF denom.'] = totalpeaks410['Total area'] / totalpeaks410['Total length']

# write modified dataframe to new txt file
totalpeaks410.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDN/410_MS_T24_GD_DN/410A_MS_T24_GD_DN50_totals.csv")

totalpeaks410.head()

,A,C,D,E,F,G,H,I,K,L,...,m-oxid,n-deam,q-deam,Total area,Total length,% C w/ carb,% M w/ oxid,% N w/ deam,% Q w/ deam,NAAF denom.
sample total,124,48,81,91,71,103,130,0,151,303,...,25,15,5,1.125793e+09,2150,1.0,0.19084,0.130435,0.131579,523624.432558


In [8]:
# use the calculated NAAF factor (in totalpeaks dataframe, above) to caluclate the NAAF 
# NAAF: normalized normalized area abundance factor

# don't have to worry here about DECOY hits messing with Area totals
# but we would with Comet results

NAAF50 = 523624.432558

# use NAAF >50% ALC to get NAAF factor
peaks410['NAAF factor'] = (peaks410['NAAF_num.'])/NAAF50

# make a dataframe that contains only what we need: sequences, AAs, PTMs
peaksAAPTM_410 = peaks410[['stripped_peptide', 'NAAF factor', 'A', 'C', 'D', 'E', 'F', 'G', 'H', 'K', 'I', 'L', \
                                'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', 'c-carb', 'm-oxid', \
                                'n-deam', 'q-deam']].copy()

# multiply the NAAF50 factor by the AA total to normalize its abundance by peak area and peptide length

peaksAAPTM_410['A-NAAF50'] = peaksAAPTM_410['A'] * peaks410['NAAF factor']
peaksAAPTM_410['C-NAAF50'] = peaksAAPTM_410['C'] * peaks410['NAAF factor']
peaksAAPTM_410['D-NAAF50'] = peaksAAPTM_410['D'] * peaks410['NAAF factor']
peaksAAPTM_410['E-NAAF50'] = peaksAAPTM_410['E'] * peaks410['NAAF factor']
peaksAAPTM_410['F-NAAF50'] = peaksAAPTM_410['F'] * peaks410['NAAF factor']
peaksAAPTM_410['G-NAAF50'] = peaksAAPTM_410['G'] * peaks410['NAAF factor']
peaksAAPTM_410['H-NAAF50'] = peaksAAPTM_410['H'] * peaks410['NAAF factor']
peaksAAPTM_410['I-NAAF50'] = peaksAAPTM_410['I'] * peaks410['NAAF factor']
peaksAAPTM_410['K-NAAF50'] = peaksAAPTM_410['K'] * peaks410['NAAF factor']
peaksAAPTM_410['L-NAAF50'] = peaksAAPTM_410['L'] * peaks410['NAAF factor']
peaksAAPTM_410['M-NAAF50'] = peaksAAPTM_410['M'] * peaks410['NAAF factor']
peaksAAPTM_410['N-NAAF50'] = peaksAAPTM_410['N'] * peaks410['NAAF factor']
peaksAAPTM_410['P-NAAF50'] = peaksAAPTM_410['P'] * peaks410['NAAF factor']
peaksAAPTM_410['Q-NAAF50'] = peaksAAPTM_410['Q'] * peaks410['NAAF factor']
peaksAAPTM_410['R-NAAF50'] = peaksAAPTM_410['R'] * peaks410['NAAF factor']
peaksAAPTM_410['S-NAAF50'] = peaksAAPTM_410['S'] * peaks410['NAAF factor']
peaksAAPTM_410['T-NAAF50'] = peaksAAPTM_410['T'] * peaks410['NAAF factor']
peaksAAPTM_410['V-NAAF50'] = peaksAAPTM_410['V'] * peaks410['NAAF factor']
peaksAAPTM_410['W-NAAF50'] = peaksAAPTM_410['W'] * peaks410['NAAF factor']
peaksAAPTM_410['Y-NAAF50'] = peaksAAPTM_410['Y'] * peaks410['NAAF factor']

# multiply the NAAF50 factor by the PTMs normalize its abundance by peak area and peptide length

peaksAAPTM_410['ccarb-NAAF50'] = peaksAAPTM_410['c-carb'] * peaksAAPTM_410['NAAF factor']
peaksAAPTM_410['moxid-NAAF50'] = peaksAAPTM_410['m-oxid'] * peaksAAPTM_410['NAAF factor']
peaksAAPTM_410['ndeam-NAAF50'] = peaksAAPTM_410['n-deam'] * peaksAAPTM_410['NAAF factor']
peaksAAPTM_410['qdeam-NAAF50'] = peaksAAPTM_410['q-deam'] * peaksAAPTM_410['NAAF factor']


# write the dataframe to a new csv
peaksAAPTM_410.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDN/410_MS_T24_GD_DN/410A_MS_T24_GD_DN50_naaf.csv")

peaksAAPTM_410.head()

,stripped_peptide,NAAF factor,A,C,D,E,F,G,H,K,...,R-NAAF50,S-NAAF50,T-NAAF50,V-NAAF50,W-NAAF50,Y-NAAF50,ccarb-NAAF50,moxid-NAAF50,ndeam-NAAF50,qdeam-NAAF50
0,SPATLNSR,0.106469,1,0,0,0,0,0,0,0,...,0.106469,0.212939,0.106469,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
1,TEELDR,0.005220,0,0,1,2,0,0,0,0,...,0.005220,0.000000,0.005220,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
2,SCK,0.073208,0,2,0,1,0,0,0,1,...,0.000000,0.146415,0.000000,0.0,0.0,0.0,0.146415,0.000000,0.0,0.0
3,SPATLNSR,0.106469,1,0,0,0,0,0,0,0,...,0.106469,0.212939,0.106469,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
4,KMLNAAK,0.040105,2,0,0,0,0,0,0,2,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.040105,0.0,0.0


In [9]:
# made a dataframe that's the sum of NAAF corrected AAs and PTMs

index = ['sample total']

data = {'NAAF': peaksAAPTM_410['NAAF factor'].sum(),
        'A': peaksAAPTM_410['A-NAAF50'].sum(),
        'C': peaksAAPTM_410['C-NAAF50'].sum(),
        'D': peaksAAPTM_410['D-NAAF50'].sum(),
        'E': peaksAAPTM_410['E-NAAF50'].sum(),
        'F': peaksAAPTM_410['F-NAAF50'].sum(),
        'G': peaksAAPTM_410['G-NAAF50'].sum(),
        'H': peaksAAPTM_410['H-NAAF50'].sum(),
        'I': peaksAAPTM_410['I-NAAF50'].sum(),
        'K': peaksAAPTM_410['K-NAAF50'].sum(),
        'L': peaksAAPTM_410['L-NAAF50'].sum(),
        'M': peaksAAPTM_410['M-NAAF50'].sum(),
        'N': peaksAAPTM_410['N-NAAF50'].sum(),
        'P': peaksAAPTM_410['P-NAAF50'].sum(),
        'Q': peaksAAPTM_410['Q-NAAF50'].sum(),
        'R': peaksAAPTM_410['R-NAAF50'].sum(),
        'S': peaksAAPTM_410['S-NAAF50'].sum(),
        'T': peaksAAPTM_410['T-NAAF50'].sum(),
        'V': peaksAAPTM_410['V-NAAF50'].sum(),
        'W': peaksAAPTM_410['W-NAAF50'].sum(),
        'Y': peaksAAPTM_410['Y-NAAF50'].sum(),
        'c-carb': peaksAAPTM_410['ccarb-NAAF50'].sum(),
        'm-oxid': peaksAAPTM_410['moxid-NAAF50'].sum(),
        'n-deam': peaksAAPTM_410['ndeam-NAAF50'].sum(),
        'q-deam': peaksAAPTM_410['qdeam-NAAF50'].sum(),
       }

totalpeaks50_NAAF = pd.DataFrame(data, columns=['NAAF', 'A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', \
                                           'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', \
                                           'W', 'Y', 'c-carb', 'm-oxid', 'n-deam', 'q-deam'], index=index)

# calculate NAAF-corrected percentage of C's with carb (should be 1.0)
totalpeaks50_NAAF['% C w/ carb.'] = totalpeaks50_NAAF['c-carb'] / totalpeaks50_NAAF['C'] 

# calculate NAAF-corrected percentage of M's that are oxidized
totalpeaks50_NAAF['% M w/ oxid'] = totalpeaks50_NAAF['m-oxid'] / totalpeaks50_NAAF['M'] 

# calculate NAAF-corrected percentage of N's that are deamidated
totalpeaks50_NAAF['% N w/ deam'] = totalpeaks50_NAAF['n-deam'] / totalpeaks50_NAAF['N'] 

# calculate NAAF-corrected percentage of N's that are deamidated
totalpeaks50_NAAF['% Q w/ deam'] = totalpeaks50_NAAF['q-deam'] / totalpeaks50_NAAF['Q'] 

# calculate NAAF summed numerator over denominator (in above cell) for all peptides in dataset i: a check
totalpeaks50_NAAF['NAAF check'] = totalpeaks50_NAAF['NAAF'] / 523624.432558

# write modified dataframe to new txt file, same name + totals
totalpeaks50_NAAF.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDN/410_MS_T24_GD_DN/410A_MS_T24_GD_DN50_naaf_totals.csv")

totalpeaks50_NAAF.head()

,NAAF,A,C,D,E,F,G,H,I,K,...,Y,c-carb,m-oxid,n-deam,q-deam,% C w/ carb.,% M w/ oxid,% N w/ deam,% Q w/ deam,NAAF check
sample total,260.319063,195.876801,9.443365,7.811375,3.818059,6.975985,13.505495,13.968364,0.0,9.157137,...,7.270405,9.443365,5.020876,1.369095,0.371529,1.0,0.309326,0.032969,0.182921,0.000497


## Export stripped peptides >50% ALC

In [11]:
##### keep only stripped peptide column 
pep50 = peaks410[["stripped_peptide"]]

# write altered dataframe to new txt file
# used header and index parameters to get rid of 'Peptide' header and the indexing

pep50.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDN/410_MS_T24_GD_DN/410A_MS_T24_GD_DN50_stripped_peptides.txt", header=False, index=False)

# made the text file into a FASTA 
!awk '{print ">"NR"\n"$0}' /home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDN/410_MS_T24_GD_DN/410A_MS_T24_GD_DN50_stripped_peptides.txt > \
/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDN/410_MS_T24_GD_DN/410A_MS_T24_GD_DN50_stripped_peptides.fas

# removing redundancy
peaks50dedup = pd.DataFrame.drop_duplicates(pep50)

print("# redundant stripped Peaks peptides >50% ALC", len(pep50))
print("# nonredundant stripped Peaks peptides >50% ALC", len(peaks50dedup))
print("average peptide length Peaks peptides >50% ALC", peaks410['stripped_length'].mean())

# count all unique peptide (modified peptides included)
# keep only peptide column >50% ALC
pep50m = peaks410[["Peptide"]]

# deduplicate
pep50mdedup = pd.DataFrame.drop_duplicates(pep50m)

print("# redundant Peaks peptides >50% ALC", len(pep50m))
print("# nonredundant Peaks peptides", len(pep50mdedup))

# check
pep50.head()

# redundant stripped Peaks peptides >50% ALC 236
# nonredundant stripped Peaks peptides >50% ALC 168
average peptide length Peaks peptides >50% ALC 9.110169491525424
# redundant Peaks peptides >50% ALC 236
# nonredundant Peaks peptides 169


,stripped_peptide
0,SPATLNSR
1,TEELDR
2,SCK
3,SPATLNSR
4,KMLNAAK
